In [26]:
from tqdm.notebook import tqdm
from cfb import models as cfb_models
import pandas as pd; DF=pd.DataFrame
import numpy as np; import requests

from utils import *
import warnings
warnings.filterwarnings("ignore")

In [27]:
# Create Season Instances
for year in range(1900,2100):
    season,c = Season.objects.get_or_create(
        year=year,defaults=dict(year=year)
    )

## Create Season/Coach/Conference Instances

In [28]:
coaches_df=pd.read_csv('_Coaches8.csv')
col_list=['SchoolPay','TotalPay','Bonus','BonusPaid','AssistantPay','Buyout']
coaches_df = DataFrameCleaner(coaches_df).keep_numeric_only(col_list=col_list)


for year in range(1900,2100):
    season,c = Season.objects.get_or_create(
        year=year,defaults=dict(year=year)
    )
season = Season.objects.get(year=2019)
def main():
    print(f'------------- FB Season: {season} ------------------')
    for tup in tqdm(coaches_df.itertuples()):
        # get or create school
        school,c=cfb_models.School.objects.get_or_create(
            name=tup.School,defaults=dict(name=tup.School)
        )
        # get or create team
        team,c=cfb_models.Team.objects.get_or_create(
            school=school,type='Football',
            defaults=dict(school=school,type='Football')
        )
        # get or create conference
        conf,c=cfb_models.Conference.objects.get_or_create(
            name=tup.Conference,defaults=dict(name=tup.Conference)
        )

        # get or create coach
        coach_name_arr = tup.Coach.strip().split(' ')
        cfn=coach_name_arr[0]; cln=coach_name_arr[1]
        coach,c = cfb_models.Coach.objects.get_or_create(
            firstName=cfn,lastName=cln,
            defaults=dict(firstName=cfn,lastName=cln)
        )

        # get or create team season
        tseason,c = TeamSeason.objects.update_or_create(
            team=team,season=season,conference=conf,coach=coach,
            defaults=dict(team=team,season=season,conference=conf,coach=coach)
        )

        cpay,c = CoachPay.objects.update_or_create(
            teamseason=tseason,base=tup.SchoolPay,total=tup.TotalPay,
            bonus=tup.Bonus,bonusPaid=tup.BonusPaid,buyout=tup.Buyout,
            defaults=dict(
                teamseason=tseason,base=tup.SchoolPay,total=tup.TotalPay,
                bonus=tup.Bonus,bonusPaid=tup.BonusPaid,buyout=tup.Buyout,        
            )
        )
#main()  

------------- FB Season: 2019 ------------------


### Scrape cfbstats.com to extract url id for each team

In [30]:
#### Get Teams URL ID from cfbstats

def translate_team_name(old_name):
    team_match_dict={
        'BYU':'Brigham Young',"Hawai'i":'Hawaii', 
        'Miami (Florida)':'Miami (Fla.)', 'SMU':'Southern Methodist', 
        'TCU':'Texas Christian', 'UAB':'Alabama at Birmingham', 
        'UCF':'Central Florida', 'UNLV':'Nevada-Las Vegas',
        'USC':'Southern California','UTEP':'Texas-El Paso', 
        'UTSA':'Texas-San Antonio','Appalachian St.':'Appalachian State',
        'Fla. Atlantic':'Florida Atlantic',"Florida Int'l":'Florida International',
        'Western Ky.':'Western Kentucky','Middle Tenn. St.':'Middle Tennessee'
    } 
    old_name = replace_abbrevs(old_name)
    if old_name in team_match_dict.keys():
        new_name = team_match_dict[old_name]    
    else:
        new_name = old_name
    return new_name

def replace_abbrevs(old_name):
    new_name=old_name
    replace_arr=[
        ['St.','State'],
        ['Ala.','Alabama'],
        ['Ga.','Georgia'],
        ['Fla.','Florida'],
        ['Ill.','Illinois'],
        ['Caro.','Carolina'],
        ['Ky.','Kentucky'],
        ['La.','Louisiana'],
        ['Mich.','Michigan'],
        ['Miss.','Mississippi'],
    ]
    for r in replace_arr:
        new_name=new_name.replace(r[0],r[1])
    return new_name
    
def get_team_home_page_dfs_from_cfbstats(team_id):
    url = f'http://www.cfbstats.com/2019/team/{team_id}/index.html'
    r=requests.get(url)
    try:
        dfs=pd.read_html(r.content)
    except ValueError:
        dfs=[]
    return dfs

def main():
    found_count=0
    target = len(coaches_df)
    for i in tqdm(range(0,1000)):
        urlId=str(i)
        dfs=get_team_home_page_dfs_from_cfbstats(i)
        if len(dfs)==0: continue
        found_count+=1
        tname = dfs[0].columns[1]
        tname = translate_team_name(tname)
        team = Team.objects.filter(school__name=tname).first()

        cfbs_id,c = CfbstatsUrlId.objects.update_or_create(
            team=team,urlId=urlId,
            defaults=dict(team=team,urlId=urlId)
        )
#main()


## ----- Get 2019 Season Stats -----

In [49]:
teamseason = TeamSeason.objects.order_by('?').first()
for teamseason in tqdm(TeamSeason.objects.all()):
    dfs = get_team_home_page_dfs_from_cfbstats(teamseason.team.cfbs.urlId)
    team_stats_df = dfs[0]
    team_stats_df = team_stats_df.rename(columns={'Unnamed: 0':'stat_str'})
    tname = team_stats_df.columns[1]
    team_stats_df = team_stats_df.rename(columns={tname:'value'})

    game_result_tdf = dfs[1]
    record_df = dfs[2]

    def get_loc(val):
        if val[0]=='@':
            loc = 'A' 
        elif val[0]=='+':
            loc = 'N'
        else:
            loc = 'H'
        return loc

    game_result_df = game_result_tdf.iloc[:-1,:].copy()
    game_result_df['loc']=game_result_df['Opponent'].map(get_loc)
    game_result_df['Opponent']=game_result_df['Opponent'].str.replace('@','').str.replace('+','')
    game_result_df['Date']=pd.to_datetime(game_result_df['Date'])
    game_result_df=game_result_df.rename(columns={'Game Time':'Game_Time'})

    print(f'----- {teamseason} ---------')
    print(f'--------------------------------------')
    print('------------- Records -------------')
    for tup in record_df.itertuples():
        desc=tup.Split
        record = tup.Record.strip().split('-')
        win=record[0]
        loss=record[1]

        ts_record,c = TeamSeasonRecord.objects.update_or_create(
            teamseason=teamseason,desc=desc,win=win,loss=loss,
            defaults=dict(teamseason=teamseason,desc=desc,win=win,loss=loss)
        )
        print(ts_record)

    print('------------- Game Results -------------')
    for tup in game_result_df.itertuples():
        date = tup.Date
        attendance = tup.Attendance
        result_str = tup.Result
        result = result_str[0]
        if result not in ['L','W']: result = 'T'
        score_str = result_str[2:]
        score1 = score_str.split('-')[0]
        score2 = score_str.split('-')[1]
        score_arr = [int(score1),int(score2)]
        if result=='W':
            score,oscore=(max(score_arr),min(score_arr))
        else:
            score,oscore=(min(score_arr),max(score_arr))

        oname=tup.Opponent.strip()\
            .replace('St.','State').replace('Ala.','Alabama')
        if oname[0].isnumeric():
            orank = oname.split(' ')[0]
            oname = " ".join(oname.split(' ')[1:])
        else:
            orank=None
        oname = translate_team_name(oname)       

        glen = str(tup.Game_Time)
        glen_arr = glen.split(':')
        hours = float(glen_arr[0])
        hour_frac = float(glen_arr[1])/60 if len(glen.split(':'))>1 else 0
        glen = hours + hour_frac
        print(glen,'GLEN')

        opponent = TeamSeason.objects.filter(
            season=teamseason.season,
            team__school__name=oname
        ).first()

        gr,c = GameResult.objects.update_or_create(
            teamseason=teamseason,
            oname=oname,
            date=date,
            defaults=dict(
                teamseason=teamseason,
                opponent=opponent,
                oname=oname,
                date=date,
                score=score,
                oscore=oscore,
                result=result,
                attendance=attendance,
                length=glen,
            )
        )
        print(gr)
    print('------------- Team Season Stats -------------')
    for tup in team_stats_df.itertuples():
        stat_str = tup.stat_str
        category=(stat_str.split(':')[0]).replace('/','per')
        desc_str=(stat_str.split(':')[1]).strip() if len(stat_str.split(':'))>1 else category


        desc_arr = desc_str.replace('/','per').split(' - ')
        value_arr = tup.value.replace('%','').split(' - ')
        ovalue_arr = tup.Opponents.replace('%','').split(' - ')


        vtups = list(zip(desc_arr,value_arr,ovalue_arr))
        for vtup in vtups:
            desc=vtup[0]
            value=vtup[1]
            ovalue=vtup[2]
            value = value.replace(',','')
            ovalue = ovalue.replace(',','')  
            if ':' in value:
                value = float(value.split(':')[0]) + float(value.split(':')[1])/60
                ovalue = float(ovalue.split(':')[0]) + float(ovalue.split(':')[1])/60

            if value=='-':value=np.nan
            if ovalue=='-':ovalue=np.nan

            
            print(category,desc,value,ovalue)
            tss,c = TeamSeasonStat.objects.update_or_create(
                teamseason=teamseason,
                category=category,
                desc=desc,
                defaults=dict(
                    teamseason=teamseason,
                    category=category,
                    desc=desc,
                    value=value,
                    ovalue=ovalue               
                )
            )
            print(tss)

----- 2019, Football, Air Force ---------
--------------------------------------
------------- Records -------------
2019, Football, Air Force, All Games, 11-2
2019, Football, Air Force, at Home, 6-0
2019, Football, Air Force, on Road/Neutral Site, 5-2
2019, Football, Air Force, vs. Conference, 7-1
2019, Football, Air Force, vs. Non-Conference, 4-1
2019, Football, Air Force, vs. Ranked (AP), 0-2
2019, Football, Air Force, vs. Unranked (AP), 11-0
2019, Football, Air Force, vs. FBS (I-A), 10-2
2019, Football, Air Force, vs. FCS (I-AA), 1-0
2019, Football, Air Force, vs. FBS Winning, 3-2
2019, Football, Air Force, vs. FBS Non-Winning, 7-0
2019, Football, Air Force, vs. BCS AQ, 2-1
2019, Football, Air Force, vs. BCS non-AQ, 8-1
2019, Football, Air Force, vs. FBS Power 5, 2-0
2019, Football, Air Force, vs. FBS non-Power 5, 8-2
2019, Football, Air Force, in August/September, 3-1
2019, Football, Air Force, in October, 3-1
2019, Football, Air Force, in November, 4-0
2019, Football, Air Force, 

2019, Football, Akron vs Northern Illinois, L 0-49
3.216666666666667 GLEN
2019, Football, Akron vs Bowling Green, L 6-35
3.0833333333333335 GLEN
2019, Football, Akron vs Eastern Michigan, L 14-42
3.05 GLEN
2019, Football, Akron vs Miami (Ohio), L 17-20
3.0833333333333335 GLEN
2019, Football, Akron vs Ohio, L 3-52
------------- Team Season Stats -------------
Scoring PointsperGame 10.5 36.3
2019, Football, Akron, Scoring-|-PointsperGame, 10.5 [opp(36.3)]
Scoring Games 12 12
2019, Football, Akron, Scoring-|-Games, 12 [opp(12)]
Scoring Points 126 435
2019, Football, Akron, Scoring-|-Points, 126 [opp(435)]
First Downs Total 155 261
2019, Football, Akron, First Downs-|-Total, 155 [opp(261)]
First Downs Rushing 51 138
2019, Football, Akron, First Downs-|-Rushing, 51 [opp(138)]
First Downs Passing 91 106
2019, Football, Akron, First Downs-|-Passing, 91 [opp(106)]
First Downs By Penalty 13 17
2019, Football, Akron, First Downs-|-By Penalty, 13 [opp(17)]
Rushing Yards per Attempt 1.75 4.53
2019

2019, Football, Alabama, Passing-|-Yards, 4449 [opp(2434)]
Passing Attempts 406 410
2019, Football, Alabama, Passing-|-Attempts, 406 [opp(410)]
Passing Completions 287 230
2019, Football, Alabama, Passing-|-Completions, 287 [opp(230)]
Passing Interceptions 6 17
2019, Football, Alabama, Passing-|-Interceptions, 6 [opp(17)]
Passing TD 49 15
2019, Football, Alabama, Passing-|-TD, 49 [opp(15)]
Total Offense Yards per Play 7.89 4.81
2019, Football, Alabama, Total Offense-|-Yards per Play, 7.89 [opp(4.81)]
Total Offense Plays 842 877
2019, Football, Alabama, Total Offense-|-Plays, 842 [opp(877)]
Total Offense Yards 6640 4217
2019, Football, Alabama, Total Offense-|-Yards, 6640 [opp(4217)]
Punt Returns Yards per Return 24.14 9.18
2019, Football, Alabama, Punt Returns-|-Yards per Return, 24.14 [opp(9.18)]
Punt Returns Returns 22 11
2019, Football, Alabama, Punt Returns-|-Returns, 22 [opp(11)]
Punt Returns Yards 531 101
2019, Football, Alabama, Punt Returns-|-Yards, 531 [opp(101)]
Punt Returns 

2019, Football, Alabama at Birmingham, Punting-|-Yards per Punt, 41.45 [opp(43.11)]
Punting Punts 69 82
2019, Football, Alabama at Birmingham, Punting-|-Punts, 69 [opp(82)]
Punting Yards 2860 3535
2019, Football, Alabama at Birmingham, Punting-|-Yards, 2860 [opp(3535)]
Interceptions Returns 7 20
2019, Football, Alabama at Birmingham, Interceptions-|-Returns, 7 [opp(20)]
Interceptions Yards 87 184
2019, Football, Alabama at Birmingham, Interceptions-|-Yards, 87 [opp(184)]
Interceptions TD 1 1
2019, Football, Alabama at Birmingham, Interceptions-|-TD, 1 [opp(1)]
Fumbles Number 15 28
2019, Football, Alabama at Birmingham, Fumbles-|-Number, 15 [opp(28)]
Fumbles Lost 8 13
2019, Football, Alabama at Birmingham, Fumbles-|-Lost, 8 [opp(13)]
Penalties Number 101 80
2019, Football, Alabama at Birmingham, Penalties-|-Number, 101 [opp(80)]
Penalties Yards 992 743
2019, Football, Alabama at Birmingham, Penalties-|-Yards, 992 [opp(743)]
Time of Possession per Game Time of Possession per Game 31.6118

2019, Football, Appalachian State, 3rd Down Conversions-|-Conversion %, 44.5 [opp(30.69)]
3rd Down Conversions Attempts 200 202
2019, Football, Appalachian State, 3rd Down Conversions-|-Attempts, 200 [opp(202)]
3rd Down Conversions Conversions 89 62
2019, Football, Appalachian State, 3rd Down Conversions-|-Conversions, 89 [opp(62)]
4th Down Conversions Conversion % 81.25 57.69
2019, Football, Appalachian State, 4th Down Conversions-|-Conversion %, 81.25 [opp(57.69)]
4th Down Conversions Attempts 16 26
2019, Football, Appalachian State, 4th Down Conversions-|-Attempts, 16 [opp(26)]
4th Down Conversions Conversions 13 15
2019, Football, Appalachian State, 4th Down Conversions-|-Conversions, 13 [opp(15)]
Red Zone Success % 91.67 84.38
2019, Football, Appalachian State, Red Zone-|-Success %, 91.67 [opp(84.38)]
Red Zone Attempts 60 32
2019, Football, Appalachian State, Red Zone-|-Attempts, 60 [opp(32)]
Red Zone Scores 55 27
2019, Football, Appalachian State, Red Zone-|-Scores, 55 [opp(27)]


...

2019, Football, Washington State, Kickoff Returns-|-Yards, 858 [opp(506)]
Kickoff Returns TD 0 1
2019, Football, Washington State, Kickoff Returns-|-TD, 0 [opp(1)]
Punting Yards per Punt 45.07 39.51
2019, Football, Washington State, Punting-|-Yards per Punt, 45.07 [opp(39.51)]
Punting Punts 30 47
2019, Football, Washington State, Punting-|-Punts, 30 [opp(47)]
Punting Yards 1352 1857
2019, Football, Washington State, Punting-|-Yards, 1352 [opp(1857)]
Interceptions Returns 10 16
2019, Football, Washington State, Interceptions-|-Returns, 10 [opp(16)]
Interceptions Yards 126 111
2019, Football, Washington State, Interceptions-|-Yards, 126 [opp(111)]
Interceptions TD 0 1
2019, Football, Washington State, Interceptions-|-TD, 0 [opp(1)]
Fumbles Number 12 17
2019, Football, Washington State, Fumbles-|-Number, 12 [opp(17)]
Fumbles Lost 9 9
2019, Football, Washington State, Fumbles-|-Lost, 9 [opp(9)]
Penalties Number 85 88
2019, Football, Washington State, Penalties-|-Number, 85 [opp(88)]
Penalt

2019, Football, West Virginia, Penalties-|-Number, 71 [opp(77)]
Penalties Yards 587 717
2019, Football, West Virginia, Penalties-|-Yards, 587 [opp(717)]
Time of Possession per Game Time of Possession per Game 28.365333333333332 31.634666666666668
2019, Football, West Virginia, Time of Possession per Game-|-Time of Possession per Game, 28.365333333333332 [opp(31.634666666666668)]
3rd Down Conversions Conversion % 34.64 39.88
2019, Football, West Virginia, 3rd Down Conversions-|-Conversion %, 34.64 [opp(39.88)]
3rd Down Conversions Attempts 179 173
2019, Football, West Virginia, 3rd Down Conversions-|-Attempts, 179 [opp(173)]
3rd Down Conversions Conversions 62 69
2019, Football, West Virginia, 3rd Down Conversions-|-Conversions, 62 [opp(69)]
4th Down Conversions Conversion % 52.63 60
2019, Football, West Virginia, 4th Down Conversions-|-Conversion %, 52.63 [opp(60)]
4th Down Conversions Attempts 19 20
2019, Football, West Virginia, 4th Down Conversions-|-Attempts, 19 [opp(20)]
4th Down 

2019, Football, Western Kentucky, 4th Down Conversions-|-Attempts, 25 [opp(36)]
4th Down Conversions Conversions 15 18
2019, Football, Western Kentucky, 4th Down Conversions-|-Conversions, 15 [opp(18)]
Red Zone Success % 78 75.76
2019, Football, Western Kentucky, Red Zone-|-Success %, 78 [opp(75.76)]
Red Zone Attempts 50 33
2019, Football, Western Kentucky, Red Zone-|-Attempts, 50 [opp(33)]
Red Zone Scores 39 25
2019, Football, Western Kentucky, Red Zone-|-Scores, 39 [opp(25)]
Field Goals Success % 60.7 72.2
2019, Football, Western Kentucky, Field Goals-|-Success %, 60.7 [opp(72.2)]
Field Goals Attempts 28 18
2019, Football, Western Kentucky, Field Goals-|-Attempts, 28 [opp(18)]
Field Goals Made 17 13
2019, Football, Western Kentucky, Field Goals-|-Made, 17 [opp(13)]
PAT Kicking Success % 97.5 96.6
2019, Football, Western Kentucky, PAT Kicking-|-Success %, 97.5 [opp(96.6)]
PAT Kicking Attempts 40 29
2019, Football, Western Kentucky, PAT Kicking-|-Attempts, 40 [opp(29)]
PAT Kicking Made

2019, Football, Western Michigan, 2-Point Conversions-|-Success %, 0 [opp(25)]
2-Point Conversions Attempts 3 4
2019, Football, Western Michigan, 2-Point Conversions-|-Attempts, 3 [opp(4)]
2-Point Conversions Made 0 1
2019, Football, Western Michigan, 2-Point Conversions-|-Made, 0 [opp(1)]
----- 2019, Football, Wisconsin ---------
--------------------------------------
------------- Records -------------
2019, Football, Wisconsin, All Games, 10-4
2019, Football, Wisconsin, at Home, 7-0
2019, Football, Wisconsin, on Road/Neutral Site, 3-4
2019, Football, Wisconsin, vs. Conference, 7-3
2019, Football, Wisconsin, vs. Non-Conference, 3-1
2019, Football, Wisconsin, vs. Ranked (AP), 3-3
2019, Football, Wisconsin, vs. Unranked (AP), 7-1
2019, Football, Wisconsin, vs. FBS (I-A), 10-4
2019, Football, Wisconsin, vs. FCS (I-AA), 0-0
2019, Football, Wisconsin, vs. FBS Winning, 6-3
2019, Football, Wisconsin, vs. FBS Non-Winning, 4-1
2019, Football, Wisconsin, vs. BCS AQ, 8-4
2019, Football, Wiscons

2019, Football, Wyoming, in August/September, 4-1
2019, Football, Wyoming, in October, 2-1
2019, Football, Wyoming, in November, 1-3
2019, Football, Wyoming, in December/January, 1-0
------------- Game Results -------------
3.5833333333333335 GLEN
2019, Football, Wyoming vs Missouri, W 37-31
3.15 GLEN
2019, Football, Wyoming vs Texas State, W 23-14
3.2333333333333334 GLEN
2019, Football, Wyoming vs Idaho, W 21-16
3.75 GLEN
2019, Football, Wyoming vs Tulsa, L 21-24
3.3666666666666667 GLEN
2019, Football, Wyoming vs Nevada-Las Vegas, W 53-17
3.0833333333333335 GLEN
2019, Football, Wyoming vs San Diego State, L 22-26
2.9 GLEN
2019, Football, Wyoming vs New Mexico, W 23-10
2.8166666666666664 GLEN
2019, Football, Wyoming vs Nevada, W 31-3
3.533333333333333 GLEN
2019, Football, Wyoming vs Boise State, L 17-20
3.533333333333333 GLEN
2019, Football, Wyoming vs Utah State, L 21-26
3.3833333333333333 GLEN
2019, Football, Wyoming vs Colorado State, W 17-7
2.716666666666667 GLEN
2019, Football, Wy